<a href="https://colab.research.google.com/github/lclem/logic_course/blob/master/docs/labs/LabZ3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAT solving in Python with Z3

We will be using Z3 programmatically in Python.
Consider the following classical logic tautologies:

\begin{align}
    \varphi_1 &\ \equiv\ p \vee \neg p & \textrm{ (excluded middle)} \\
    \varphi_2 &\ \equiv\ ((p \to q) \to p) \to p & \textrm{ (Pierce's law)}.
\end{align}
 
The Z3 code below (via its convenient Python interface) checks that the formulas above are tautologies by trying to find a satisfying assignment to their negation.

In [0]:
%pip install z3-solver

from z3 import *

# create a fresh propositional variable uniquely identified by its name 'p'
p = Bool('p')

# excluded middle
solve(Not(Or(p, Not(p))))

# Pierce's law
q = Bool('q')
solve(Not(Implies(Implies(Implies(p, q), p), p)))

#Other useful connectives:
#And

## Exercise (tautologies)

Prove that the following propositional formula is a classical tautology
by checking that its negation is unsatisfiable:
\begin{align}
    (p \land q \to r) \leftrightarrow (p \to q \to r) \\    
\end{align}
What about the following Łukasiewicz's formula?
\begin{align}
((p \to q) \to r) \to (r \to p) \to s \to p.
\end{align}

In [0]:
r = Bool('r')
s = Bool('s')

## Example (long conjunctions and disjunctions)

We can conveniently use arrays to represent long conjunctions `And` and disjunctions `Or`.
For instance the next cell implements the following formula:
\begin{align}
    x_1 \wedge x_2 \wedge \cdots \wedge x_n.
\end{align}

In [0]:
n = 10
x = [Bool('x' + str(k)) for k in range(n)]
phi = And([x[k] for k in range(n)])
print(phi)
solve(phi)

## Exercise (The SCS problem)

Let $\Sigma = \{0, 1\}$ be a finite alphabet.
The *shortest common substring* problem (SCS) has as input a set of finite strings
$S = \{w_1, \dots, w_m\}$ with $w_i \in \Sigma^*$
and a bound $n$, and asks to determine whether there is a string $w \in \Sigma^n$ of length $n$
s.t. every $w_i$ is a (*contiguous*) substring of $w$. The SCS problem is NP-complete.
1. Encode the SCS problem as a SAT problem and use Z3 to solve it.
2. Can you reconstruct a solution $w$ from the Z3 model?

In [0]:
# input example
S = ["001010011101", "1001010101011", "101101010101010",
     "10111011001010101010", "10110101011010101010", "10110101101010101010"]
m = len(S)
n = 75 # tight
# 12 + 13 + 15 + 20 + 20 + 20 = 100
# students reported 74 is a solution

# how to declare a list of n variables x0, ..., x(n-1)
x = [ Bool('x' + str(k)) for k in range(n) ]

# Solution for S: 001010011101101011010101010100101010101110110010101010101101010110101010100

# this procedure returns an assignment (as a dictionary) if any exists
def model(phi):
    
    # create a SAT instance
    s = Solver()
    s.add(phi)

    # return a satisfying assignment
    return s.model() if s.check() == sat else []

## Exercise (All satisfying assignments)
The Z3 solver returns some satisfying assignment, just in case it exists. How can we use Z3 in order to construct all satisfying assignments? Write a Python program that returns all satisfying assignments of a given SAT instance and test it on the Pigeonhole formulas from the previous exercise.

In [0]:
# this procedure returns an assignment (as a dictionary) if any exists
def model(phi):
    
    # create a SAT instance
    s = Solver()
    s.add(phi)

    # return a satisfying assignment as a dictionary
    return s.model() if s.check() == sat else []

p = Bool("p")
q = Bool("q")
r = Bool("r")
phi = Or(And(p, Not(q)), r)
m = model(phi)
print(m[p])
print(m[q])
print(m[r])

# Pro tip 1: can enumerate all variables by looking at the keys of m
# Pro tip 2: if x is a key of m,
# the corresponding Z3 variable can be reconstructed with "Bool(str(x))"
# (it should be just "x", but so is life)
vars = [Bool(str(x)) for x in m]

## Exercise (Phase transition)

In this exercise we explore the phenomenon of *phase transition* for the $k$-SAT problem, where $k$ is the size of each clause.
Let $n$ be the number of variables $X = \{x_1, \dots, x_n\}$ and let $m$ be the number of clauses.
A random $k$-SAT instance is obtained by producing each of the $m$ clauses according to the following process:

* Extract $k$ variables without replacement from $X$ and determine independently and uniformly whether each variable appears positively or negatively.

Fix $k = 3$, $n = 10$,
and let $p_{m/n}$ be the probability that a SAT instance randomly generated as above is satisfiable.
1. For a fixed $m$, compute an estimate $\hat p_{m / n}$ by sampling $N = 100$ instances as above.
2. Plot the estimates $\hat p_{m / n}$ as a function of $m \in \{1, \dots, 10 * n \}$.
3. What is an "interesting" interval for $m$? Refine the interval above if necessary.

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import random

# number of variables
n = 10

# size of a clause
k = 3

# number of samples per point
N = 100

variables = [Bool('x' + str(i)) for i in range(n)]

# Hints:
# np.random.choice(list) returns a random element from list
# np.random.choice(list, k, replace=False) returns k random elements from list *without replacement*
# np.mean(list) computes the average of the numbers in the list
# plt.plot(list) generates a plot from a list of values
# plt.show() displays the plot